In [ ]:
%load_ext autoreload
%autoreload 2

import time
import os
import argparse
import cProfile
import pstats
import logging

import numpy as np
import astropy.io.fits as pyfits

In [ ]:

import howfsc
import eetc
from howfsc.control.cs import ControlStrategy
from howfsc.model.mode import CoronagraphMode
from howfsc.util.loadyaml import loadyaml

from corgihowfsc.utils.howfsc_initialization import get_args, load_files
from corgihowfsc.sensing.DefaultEstimator import DefaultEstimator
from corgihowfsc.sensing.DefaultProbes import DefaultProbes
from corgihowfsc.utils.contrast_nomalization import EETCNormalization
from corgihowfsc.utils.corgisim_gitl_frames import GitlImage
from corgihowfsc.gitl.nulling_gitl import nulling_gitl


In [ ]:
eetc_path = os.path.dirname(os.path.abspath(eetc.__file__))
howfscpath = os.path.dirname(os.path.abspath(howfsc.__file__))


defjacpath = os.path.join(os.path.dirname(howfscpath), 'jacdata')

defjacpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\cgi-howfsc'


In [ ]:
args = get_args(jacpath=defjacpath)

# Initialize variables etc

otherlist = []
abs_dm1list = []
abs_dm2list = []
framelistlist = []
scalelistout = []
camlist = []

# User params
niter = args.niter
mode = args.mode
isprof = args.profile
logfile = args.logfile
fracbadpix = args.fracbadpix
nbadpacket = args.nbadpacket
nbadframe = args.nbadframe
fileout = args.fileout
stellar_vmag = args.stellarvmag
stellar_type = args.stellartype
stellar_vmag_target = args.stellarvmagtarget
stellar_type_target = args.stellartypetarget
jacpath = args.jacpath

modelpath, cfgfile, jacfile, cstratfile, probefiles, hconffile, n2clistfiles = load_files(args, howfscpath)

# Initialize default probes class
probes = DefaultProbes('default')

# # Get DM lists
# probefiles = {}
# probefiles[0] = probe0file
# probefiles[2] = probe1file
# probefiles[1] = probe2file
# dm1_list, dm2_list = probing.get_dm_probes(cfg, probefiles,
#                       scalelist=[0.3, 0.3, 0.3, -0.3, -0.3, -0.3])
#
# # Get probe amplitude lists (this step is slow)
# plist, other = probing.get_probe_ap(cfg, dm1_list, dm2_list)



In [ ]:
# cfg
cfg = CoronagraphMode(cfgfile)

# hconffile
hconf = loadyaml(hconffile, custom_exception=TypeError)

cstrat = ControlStrategy(cstratfile)
estimator = DefaultEstimator()

normalization_strategy = EETCNormalization()

# imager = corgisimImager()
# imager = compactImager()

imager = GitlImage(
    cfg=cfg,         # Your CoronagraphMode object
    cstrat=cstrat,   # Your ControlStrategy object  
    hconf=hconf,      # Your host config with stellar properties
    backend='cgi-howfsc', 
    cor=mode
)



In [ ]:
nulling_gitl(cstrat, estimator, probes, normalization_strategy, imager, cfg, args, hconf, modelpath, jacfile, probefiles, n2clistfiles)
